In [119]:
class Operad:
    def __init__(self):
        self.colors = {}
        self.operations = {}

    def add_color(self, name, prop={}):
        if t := self.colors.get(name):
            self.colors[name] = (t, prop)
        else:
            self.colors[name] = prop
        return name, t

    def add_operation(self, name, prop={}):
        self.operations[name] = prop
        return name


class Tree:
    def __init__(self, operation, trunk, branches, operad):

        trunk, parent = operad.add_color(trunk, self)
        self.trunk = trunk
        if parent:
            parent.branches[trunk] = self
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        self.branches = {
            operad.add_color(branch, self)[0]: uTree(branch, depth=self.depth)
            for branch in branches
        }
        self.node = operad.add_operation(operation, self)

    def print_edges(self):
        return "".join(
            [f"{str(self.trunk)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_edges()}\n"
                for branch in self.branches.values()
            ]
        )

    def print_nodes(self):
        return "".join(
            [f"{str(self.node)}\n"]
            + [
                "\t" * (self.depth + 1) + f"{branch.print_nodes()}\n"
                for branch in self.branches.values()
            ]
        )

    def __str__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"

    def __repr__(self):
        return f"{self.node}({','.join(self.branches.keys())};{self.trunk})"


class uTree(Tree):
    def __init__(self, name, depth):
        self.trunk = name
        self.branches = {}
        self.node = None
        self.depth = depth


In [140]:
import re
from functools import cmp_to_key

def string_to_tree_space(string, operad):

    operads = []
    operations = string.split("|")
    operations = sorted(operations, key=cmp_to_key(compare))
    regex = r"(.*)\((.*)\)"
    for operation in operations:
        match = re.match(regex, operation)
        if not match:
            raise RuntimeError(f"Operation not defined correctly {operation}")
        operation, parameters = match.groups()
        branches, trunk = parameters.split(";")
        operads.append(Tree(operation, trunk, branches.split(","), operad))
    return  operads[0], operad


def tree_space_to_string(tree_space):
    _, operad = tree_space
    return "|".join(str(tree) for tree in operad.operations.values())

def _recursive_str(tree, s):
    s.append(str(tree))
    for branch in tree.branches.values():
        if not isinstance(branch, uTree):
            _recursive_str(branch, s)
    return s

def tree_to_string(tree):
    operations = sorted(_recursive_str(tree, []), key=cmp_to_key(compare))
    return "|".join(operations)

def extract_compare(item):
    regex = r"(.*)\((.*)\)"
    match = re.match(regex, item)
    if not match:
        raise RuntimeError(f"Operation not defined correctly {item}")
    _, parameters = match.groups()
    branches, trunk = parameters.split(";")
    return trunk, branches.split(",")
    
def compare(item1, item2):
    item1 = extract_compare(item1)
    item2 = extract_compare(item2)
    
    if not item1 or not item2:
        return 0
    if item1[0] in item2[1]:
        return 1
    elif item2[0] in item1[1]:
        return -1
    else:
        return 0



S = string_to_tree_space("o0w(1,2;0)", Operad())
T = string_to_tree_space("o0b(b,c;a)|o1b(d;b)", Operad())
# S = string_to_tree_space("0W(1;0)|1W(2,3;1)", Operad())
# T = string_to_tree_space("0B(b,d;a)|1B(c;b)|2B(e;d)", Operad())

# print(tree_space_to_string(S))
# print(tree_to_string(S[0]))
# print(tree_space_to_string(T))
# print(tree_to_string(T[0]))
# print(S[0].print_nodes())

In [142]:
from copy import deepcopy
from collections import defaultdict


class DualTree:
    def __init__(self, S, T):
        self.S_og_space = S
        self.T_og_space = T
        self.tree = S[0]
        self.T = T[0]
        self.add_color_base(self.tree, self.T.trunk)
        self.str = tree_to_string(self.tree)

    def add_color_edge(self, T, c, depth):
        T.trunk = f"{c}-{T.trunk}"
        T.branches = self.rename_keys(T, c, True)
        T.depth += depth
        for Ti in T.branches.values():
            self.add_color_edge(Ti, c, depth)
        return T

    def rename_keys(self, S, c, inv=False):
        aux = {}
        for i, Si in S.branches.items():
            if not inv:
                aux[f"{i}-{c}"] = Si
            else:
                aux[f"{c}-{i}"] = Si
        return aux

    def add_color_base(self, S, c):
        S.trunk = f"{S.trunk}-{c}"
        S.branches = self.rename_keys(S, c)
        for i, Si in S.branches.items():
            if isinstance(Si, uTree):
                S.branches[i] = self.add_color_edge(
                    deepcopy(self.T), i.split("-")[0], S.depth + 1
                )
            else:
                self.add_color_base(Si, c)

    def find_percolant_branches(self, S=None, found=[]):
        if not S:
            S = self.tree
        if S.node in self.S_og_space[1].operations:
            if any(
                branch.node in self.T_og_space[1].operations
                for branch in S.branches.values()
            ):
                found.append(S)

        for Si in S.branches.values():
            self.find_percolant_branches(Si, found)
        return found


class ShuffleLattice:
    def __init__(self, S, T):
        self.S_og_space = deepcopy(S)
        self.T_og_space = deepcopy(T)
        self.initial_tree = DualTree(deepcopy(S), deepcopy(T))
        self.skeleton = defaultdict(list)
        self.initialize_skeleton()

    def initialize_skeleton(self):

        self.skeleton[self.initial_tree.str] = []

    def generate_shuffles(self):
        queue = []
        queue.append(self.initial_tree)

        while len(queue):
            tree = deepcopy(queue.pop())
            for location in tree.find_percolant_branches():
                new_tree = self.make_percolation(tree, location)
                fingerprint = new_tree
                if not fingerprint in self.skeleton:
                    print("1")
                    # queue.append(new_tree)
                self.skeleton[fingerprint].append(tree.str)

    def make_percolation(self, tree, location):
        operations = tree.str.split("|")
        branches = list(location.branches.values())
        new_node = branches[0].node
        old_node = location.node
        to_change = [str(location)] + [str(branch) for branch in branches]
        old_operation = self.S_og_space[1].operations[old_node]
        new_operation = self.T_og_space[1].operations[new_node]
        
        morfed = []
        for _, c in new_operation.branches.items():
            morfed.append(f"{old_operation.node}({','.join(map(lambda x: x+'-'+c.trunk, old_operation.branches.keys()))};{old_operation.trunk}-{c.trunk})")
        morfed.append(f"{new_operation.node}({','.join(map(lambda x: old_operation.trunk+'-'+x, new_operation.branches.keys()))};{old_operation.trunk}-{new_operation.trunk})")

        for branch in to_change:
            operations.remove(branch)
        operations += morfed
        operations = sorted(operations, key=cmp_to_key(compare))
        new_tree = "|".join(operations)
        print(new_tree)
        return new_tree


shl = ShuffleLattice(S, T)
# shl.initial_tree.find_percolant_branches()
shl.generate_shuffles()
# print(shl.skeleton)
print(tree_to_string(shl.initial_tree.tree))
print("o0b(0-b,0-c;0-a)|o0w(1-b,2-b;0-b)|o0w(1-c,2-c;0-c)|o1b(1-d;1-b)|o1b(2-d;2-b)")

o0w(1-b,2-b;0-b)|o1b(1-d;1-b)|o1b(2-d;2-b)|o0b(0-b,0-c;0-a)|o0w(1-c,2-c;0-c)
1


AttributeError: 'str' object has no attribute 'find_percolant_branches'

In [118]:
s = string_to_tree_space("o0b(1-b,1-c;1-a)|o0b(2-b,2-c;2-a)|o0w(1-a,2-a;0-a)|o1b(1-d;1-b)|o1b(2-d;2-b)", Operad())

print(s[0].print_edges())

1-a
	1-c

	1-b
		1-d



